# Decoding Behavior from Behavior

### Import

In [1]:
# FISSA: Signal Extraction & Source-Separation
# Load this module to organize loading our trace data
from AnalysisModules.FissaAnalysis import FissaModule

# CASCADE: Spike Inference
# Load this module to organize loading our inference data
from AnalysisModules.CascadeAnalysis import CascadeModule

# Behavioral Decoder
# Load this module to organize encoding-decoding analyses
# Interfaces Neural Decoding Package from Joshua Glaser via Kording Lab.
# Interfaces with State-Space Modeling (SSM / SSM-jax) Package from Scott Linderman Lab
# Interfaces with an annotated implementation of
# Manifold Inference from Neural Dynamics Package (MIND) from David Tank Lab
# Annotated/Implemented by Quentin RV Ferry via Tonegawa Lab
# Interfaces with PySindy Package from the Dynamics Lab at UW
from AnalysisModules.DecodingAnalysis import DecodingModule # Not needed for this nb, as imported inline

# Visualization Packages
import matplotlib
#%matplotlib inline
matplotlib.use('Qt5Agg')

# General Packages
import numpy as np

C:\ProgramData\Anaconda3\envs\Calcium-Imaging-Analysis-Pipeline\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


	YAML reader installed (version 0.17.17).


Using TensorFlow backend.


	Keras installed (version 2.3.1).
	Tensorflow installed (version 2.3.0).


### Load Data

In [2]:
# Instantiate Traces & Associated Data
Data = FissaModule(data_folder="H:\\DEM_Excitatory_Study\\DEM2\\Retrieval\\Imaging\\10Hz", index_path="H:\\DEM_Excitatory_Study\\DEM2\\Retrieval\\Imaging\\10Hz\\NeuronalIndex.csv")
# Initialize
Data.initializeFissa()
# Load
Data.loadFissaPrep()
Data.loadFissaSep()
Data.loadProcessedTraces()

# Instantiate Firing Rates & Associated Data
Data.Cascade = CascadeModule(Data.ProcessedTraces.merged_dFoF_result, Data.frame_rate, model_folder="C:\\ProgramData\\Anaconda3\\envs\\suite2p\\Pretrained_models")
Data.Cascade.loadSpikeProb(load_path=Data.output_folder)
Data.Cascade.loadSpikeInference(load_path=Data.output_folder)
Data.Cascade.loadProcessedInferences(load_path=Data.output_folder)

Initialized Fissa
Loading Fissa Preparation...
Finished Loading Fissa Prep
Loading Fissa Separation...
Finished Loading Fissa Sep
Loading Processed Traces...
Finished Loading Processed Traces.
Loading Spike Probabilities from Load Path...
Finished Loading Spike Probabilities
Loading Spike Times from Load Path...
Loading Discrete Approximations from Load Path...
Loading Processed Inferences...
Finished Loading Processed Traces.


In [3]:
# Wiener Filter
from AnalysisModules.DecodingAnalysis import WienerFilter
# Instance
Data.WF = WienerFilter(NeuralData=Data.Cascade.ProcessedInferences.firing_rates, FeatureDataFile="C:\\ProgramData\\Anaconda3\\envs\\Calcium-Imaging-Analysis-Pipeline\\DEM2_RET_FeatureList.csv")
# Pick Single Feature
Data.WF.label_data = Data.WF.feature_data[5, :]
# Set FR == 0 when NaN
Data.WF.neural_data[np.isnan(Data.WF.neural_data)]=0
Data.WF.splitData()
# Fit
Data.WF.fitModel(fit_intercept=True)
# Assess on Training
Data.assessModel()
# Extended Assessment
Data.WF.commonAssessment()

Instanced Wiener Filter
Splitting data in training & testing sets
Data splits are: 80.0% training vs 20.0% testing
Fitting Wiener Filter...
Finished


ValueError: Found input variables with inconsistent numbers of samples: [19701, 19701, 10]